In [1]:
import os
os.environ["BRAINTRUST_API_KEY"]="sk-MbNKNf5DNKYncJDMpooa7SJr0g0aF2Xl4uk0JfrtqIp333GT"

In [2]:
import pandas as pd
# df = pd.read_csv("./sample_rag_outputs/ibis-healthcare-social-assistance_llama-parse_gpt-4o-mini.csv")
df = pd.read_csv("./sample_rag_outputs/ibis-healthcare-social-assistance_llm-sherpa_gpt-4o-mini.csv")



In [12]:
print(df.shape)

(25, 8)


In [3]:
df.head()

,question,answer,source type,answer type,page,file,explanation,rag_model_response
0,What types of enterprises are included in the ...,The Healthcare and Social Assistance sector in...,text,other,5,62 Healthcare and Social Assistance in the US ...,Understanding the scope of the Healthcare and ...,The Healthcare and Social Assistance sector in...
1,How is telemedicine defined in the context of ...,Telemedicine is defined as an application of c...,text,other,5,62 Healthcare and Social Assistance in the US ...,Identifying key technological trends like tele...,Telemedicine is defined as an application of c...
2,What are Health Insurance Exchanges in the US ...,Health Insurance Exchanges are sets of state-r...,text,other,5,62 Healthcare and Social Assistance in the US ...,Understanding the structure of health insuranc...,Health Insurance Exchanges in the US healthcar...
3,What services are included in the Healthcare a...,The Healthcare and Social Assistance industry ...,text,other,6,62 Healthcare and Social Assistance in the US ...,Understanding the scope of services in the ind...,The Healthcare and Social Assistance industry ...
4,Who are some of the major companies operating ...,Major companies in the industry include Hca He...,text,other,6,62 Healthcare and Social Assistance in the US ...,Identifying key players helps in understanding...,The context does not specify particular compan...


In [4]:
# change the column names of question to input and answer to expected
df.rename(columns={"question": "input", "answer": "expected"}, inplace=True)
df.head()

,input,expected,source type,answer type,page,file,explanation,rag_model_response
0,What types of enterprises are included in the ...,The Healthcare and Social Assistance sector in...,text,other,5,62 Healthcare and Social Assistance in the US ...,Understanding the scope of the Healthcare and ...,The Healthcare and Social Assistance sector in...
1,How is telemedicine defined in the context of ...,Telemedicine is defined as an application of c...,text,other,5,62 Healthcare and Social Assistance in the US ...,Identifying key technological trends like tele...,Telemedicine is defined as an application of c...
2,What are Health Insurance Exchanges in the US ...,Health Insurance Exchanges are sets of state-r...,text,other,5,62 Healthcare and Social Assistance in the US ...,Understanding the structure of health insuranc...,Health Insurance Exchanges in the US healthcar...
3,What services are included in the Healthcare a...,The Healthcare and Social Assistance industry ...,text,other,6,62 Healthcare and Social Assistance in the US ...,Understanding the scope of services in the ind...,The Healthcare and Social Assistance industry ...
4,Who are some of the major companies operating ...,Major companies in the industry include Hca He...,text,other,6,62 Healthcare and Social Assistance in the US ...,Identifying key players helps in understanding...,The context does not specify particular compan...


In [5]:
# select the columns "input" ,"expected", "page", "file", "source type", "answer type" from the dataframe
labelled_df = df[["input", "expected", "page", "file", "source type", "answer type"]]

# modify the input column to add the prefix row number from the index to the input
labelled_df["input"] = labelled_df.apply(lambda x: f"{x.name}> {x.input}", axis=1)
labelled_df.head()


/var/folders/9g/7kc65t_s0bgf2v24qc1f5tsw0000gn/T/ipykernel_54447/2608455778.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labelled_df["input"] = labelled_df.apply(lambda x: f"{x.name}> {x.input}", axis=1)


,input,expected,page,file,source type,answer type
0,0> What types of enterprises are included in t...,The Healthcare and Social Assistance sector in...,5,62 Healthcare and Social Assistance in the US ...,text,other
1,1> How is telemedicine defined in the context ...,Telemedicine is defined as an application of c...,5,62 Healthcare and Social Assistance in the US ...,text,other
2,2> What are Health Insurance Exchanges in the ...,Health Insurance Exchanges are sets of state-r...,5,62 Healthcare and Social Assistance in the US ...,text,other
3,3> What services are included in the Healthcar...,The Healthcare and Social Assistance industry ...,6,62 Healthcare and Social Assistance in the US ...,text,other
4,4> Who are some of the major companies operati...,Major companies in the industry include Hca He...,6,62 Healthcare and Social Assistance in the US ...,text,other


In [11]:
print(labelled_df.shape)

(25, 6)


In [6]:
# convert the labelled dataframe to list of dictionaries
labelled_data = labelled_df.to_dict(orient="records")
labelled_data[:2]

[{'input': '0> What types of enterprises are included in the Healthcare and Social Assistance sector in the United States?',
  'expected': 'The Healthcare and Social Assistance sector includes hospitals, ambulatory service providers, nursing and residential care facilities, counselors, social workers, family and welfare services, and natural disaster and emergency relief services.',
  'page': 5,
  'file': '62 Healthcare and Social Assistance in the US Industry Report.pdf',
  'source type': 'text',
  'answer type': 'other'},
 {'input': '1> How is telemedicine defined in the context of the Healthcare and Social Assistance industry?',
  'expected': 'Telemedicine is defined as an application of clinical medicine whereby medical information is transferred through interactive audiovisual media for the purpose of consulting and remote medical procedures or examinations.',
  'page': 5,
  'file': '62 Healthcare and Social Assistance in the US Industry Report.pdf',
  'source type': 'text',
  'an

In [7]:
from braintrust import Eval
 
from autoevals import Factuality
from autoevals.ragas import *



def my_task(input):
    row_number = int(input.split("> ")[0])
    return df.loc[row_number, "rag_model_response"]
 

my_task("0> What is the name of the organization that provides healthcare and social assistance services to the community?")


'The Healthcare and Social Assistance sector in the United States includes enterprises such as hospitals, ambulatory service providers, nursing and residential care facilities, counselors, social workers, family and welfare services, and organizations providing natural disaster and emergency relief services.'

In [9]:
Eval(
    "ibis-healthcare-social-assistance_llama-parse_gpt-4o-mini",  # Replace with your project name
    data=labelled_data,
    task=my_task,
    scores=[Factuality, AnswerSimilarity, AnswerCorrectness],
)

<Task pending name='Task-228' coro=<Eval.<locals>.run_to_completion() running at /Users/mbajaj/anaconda/envs/multiplai/lib/python3.10/site-packages/braintrust/framework.py:536>>

Experiment users/mohit/rag-1727930503 is running at https://www.braintrust.dev/app/Omega/p/ibis-healthcare-social-assistance_llama-parse_gpt-4o-mini/experiments/users%2Fmohit%2Frag-1727930503
ibis-healthcare-social-assistance_llama-parse_gpt-4o-mini (data): 25it [00:00, 12775.05it/s]


ibis-healthcare-social-assistance_llama-parse_gpt-4o-mini (tasks):   0%|          | 0/25 [00:00<?, ?it/s]


=========================SUMMARY=========================
users/mohit/rag-1727930503 compared to users/mohit/rag-1727930135:
64.62% (-02.79%) 'AnswerCorrectness'   score	(12 improvements, 13 regressions)
71.20% (-05.84%) 'EmbeddingSimilarity' score	(12 improvements, 13 regressions)
44.00% (-19.20%) 'Factuality'          score	(3 improvements, 8 regressions)

0.06s (-00.26%) 'duration'         	(11 improvements, 13 regressions)
0tok (-) 'prompt_tokens'    	(0 improvements, 0 regressions)
0tok (-) 'completion_tokens'	(0 improvements, 0 regressions)
0tok (-) 'total_tokens'     	(0 improvements, 0 regressions)

See results for users/mohit/rag-1727930503 at https://www.braintrust.dev/app/Omega/p/ibis-healthcare-social-assistance_llama-parse_gpt-4o-mini/experiments/users%2Fmohit%2Frag-1727930503
